In [9]:
exec(open("initKeras.py").read())

In [10]:
# model numbers to retrain, set to false or blank if you want to retrain all the models in DoodleModels. Strictly a list of integers
trainOnlyModels = []
saveModels = True
# #data preparation
inputDir = "C:\\tim209\\filtered by category\\output20k10\\*.npy"
modelDirectory = "F:\\myProjects\\tim209\\doodle\\savedModels\\"
modelGraphDirectory = "F:\\myProjects\\tim209\\doodle\\experiment results\\"

In [11]:
dataUtils = dataUtils( inputDir )
dataUtils.process()
(images, labels, classToLevels ) = dataUtils.getData()
(imagesR, labelsR, classToLevelsR ) = dataUtils.getRandomizedData()
images = dataUtils.reshapeDataForKeras( images )
imagesR = dataUtils.reshapeDataForKeras( imagesR )

Processing aircraft carrier C:\tim209\filtered by category\output10k10\aircraft carrier.npy
Observation #: 20000
label hot vectors #: 20000
Processing airplane C:\tim209\filtered by category\output10k10\airplane.npy
Observation #: 20000
label hot vectors #: 20000
Processing alarm clock C:\tim209\filtered by category\output10k10\alarm clock.npy
Observation #: 20000
label hot vectors #: 20000
Processing ambulance C:\tim209\filtered by category\output10k10\ambulance.npy
Observation #: 20000
label hot vectors #: 20000
Processing angel C:\tim209\filtered by category\output10k10\angel.npy
Observation #: 20000
label hot vectors #: 20000
Processing animal migration C:\tim209\filtered by category\output10k10\animal migration.npy
Observation #: 20000
label hot vectors #: 20000
Processing ant C:\tim209\filtered by category\output10k10\ant.npy
Observation #: 20000
label hot vectors #: 20000
Processing anvil C:\tim209\filtered by category\output10k10\anvil.npy
Observation #: 20000
label hot vectors

In [12]:
categoriesLen = labels.shape[1]
categoriesLen

10

In [6]:
doodleModels = DoodleModels(images, labels)

train X: (160000, 28, 28, 1)
test X: (40000, 28, 28, 1)
train Y: (160000, 10)
test Y: (40000, 10)


In [8]:
histories = []
suffixFile = "_cat" + str( categoriesLen ) + "_e" + str( epochSize ) + "_b" + str( batchSize )
for i in range(5):
    
    modelNo = i + 1
    
    #1 filter if needed
    if type( trainOnlyModels ) == list and len( trainOnlyModels ) > 0 and modelNo not in trainOnlyModels:
        print( "Skipping model #", modelNo )
        continue
        
    #2 get and train model
    model = doodleModels.getModel( modelNo )
    testLoss, testAcc = doodleModels.evaluateModel( model )
    histories.append( ( model, model.history, testLoss, testAcc ) )
    
    #3 save model performance plot
    plotSavePath = modelGraphDirectory + model.name + suffixFile + ".png"
    doodleModels.plotModelTrainPerformance( model, plotSavePath )
    
    #4 save trained model
    if saveModels == True:
        modelPath = modelDirectory + model.name + suffixFile + ".h5"
        doodleModels.saveModelAndHistory( model, modelPath )


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                9248      
__________

AttributeError: 'Sequential' object has no attribute 'history'

In [ ]:
#print all model graph
plt.close()
colorPalette = sns.color_palette("hls", len( histories ) )
colorIndex = 0
for ( model, history, testLoss, testAcc ) in histories:
    epochX = np.arange( len( history.epoch ) ) + 1
    plt.figure( figsize= (4 ,4))
    plt.plot( epochX, history.history['categorical_accuracy'], color = colorPalette[ colorIndex] , label = model.name + " - accuracy" )
    plt.plot( epochX, history.history['loss'], color = colorPalette[ colorIndex], label = model.name + " - loss", linestyle = '--' )

    plt.title( 'Epoch vs Test - All Models' )
    plt.xlabel( "Epoch" )
    plt.ylabel( "amount" )
    plt.xticks( epochX )
    plt.legend()
    colorIndex = colorIndex + 1

plt.savefig( modelGraphDirectory + "all-models-" + datetime.now().strftime( "%Y-%M-%d %Hh%Mm%Ss" ) )